# Tools

### Fix imports

In [ ]:
import sys
import os
# Add the parent directory to sys.path so 'experiments' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
REPOSITORY_FOLDER = "repositories"

## Github

#### Clone a repository based on url

In [ ]:
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil



@tool("git_clone")
def git_clone_tool(
    repo_url: str,
    dest: str,
    branch: Optional[str] = None,
    overwrite: bool = False,
) -> Dict[str, Any]:
    """
    Clone a Git repository into ./repositories/{dest} using GitPython.

    Args:
        repo_url: HTTPS or SSH URL of the repository.
        dest: Name of the destination folder for the clone inside ./repositories/.
        branch: Optional branch to check out.
        overwrite: If True, overwrite existing destination folder.
    Returns:
        A dict with success (bool), dest (str), and error/stdout messages.
    """

    try:
        # Ensure repositories/ root exists
        root_dir = os.path.join(os.getcwd(), REPOSITORY_FOLDER)
        os.makedirs(root_dir, exist_ok=True)

        # Full destination path inside repositories/
        full_dest = os.path.join(root_dir, dest)

        # Handle overwrite
        if os.path.exists(full_dest):
            if overwrite:
                shutil.rmtree(full_dest)
            else:
                return {"success": False, "error": f"Destination {full_dest} already exists."}


        # Clone options
        kwargs = {}
        if branch:
            kwargs["branch"] = branch
            full_dest = f"{full_dest}/{branch}"

        os.makedirs(full_dest, exist_ok=True)
        repo = Repo.clone_from(repo_url, full_dest, **kwargs)

        return {
            "success": True,
            "dest": full_dest,
            "branch": repo.active_branch.name if not repo.head.is_detached else "detached",
            "error": None,
        }
    except GitCommandError as e:
        return {"success": False, "dest": dest, "error": str(e)}
    except Exception as e:
        return {"success": False, "dest": dest, "error": str(e)}

#### Add Github MCP server based on url

In [7]:
from typing import Dict, Any
from langchain.tools import tool
from experiments.utils.mcp_client_factory import load_mcp_config, save_mcp_config, create_mcp_client_from_config


@tool("add_mcp_server")
def add_mcp_server_tool(
    name: str,
    url: str,
    transport: str = "streamable_http",
) -> Dict[str, Any]:
    """
    Add a new MCP server to the configuration.
    
    Args:
        name: Name identifier for the MCP server
        url: URL of the MCP server
        transport: Transport type (default: "streamable_http")
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be  
    """
    try:
        # Load existing config
        config = load_mcp_config()
        
        # Add new server
        config[name] = {
            "url": url,
            "transport": transport
        }
    
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Added MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }
    
@tool("add_github_repository_as_mcp_server")
def add_github_repository_as_mcp_tool(repo_url: str, server_name: str) -> Dict[str, Any]:
    """
    Add a GitHub repository as an MCP server using gitmcp.io.
    
    Args:
        repo_url: GitHub repository URL (e.g., https://github.com/owner/repo)
        server_name: Name identifier for the MCP server
    Returns:
        Dict with success status and current config
    """
    # Extract the repository path from the GitHub URL
    if "github.com/" in repo_url:
        # Extract everything after github.com/
        repo_path = repo_url.split("github.com/", 1)[1]
        # Remove .git suffix if present
        if repo_path.endswith(".git"):
            repo_path = repo_path[:-4]
        gitmcp_url = f"https://gitmcp.io/{repo_path}"
    else:
        raise ValueError("Invalid GitHub repository URL")
    
    tool_input = {
        "name": server_name,
        "url": gitmcp_url,
    }

    return add_mcp_server_tool.invoke(tool_input)

    

@tool("remove_mcp_server")
def remove_mcp_server_tool(
    name: str,
) -> Dict[str, Any]:
    """
    Remove an MCP server from the configuration.
    
    Args:
        name: Name identifier of the MCP server to remove
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be 
    """
    try:
        config = load_mcp_config()
        
        if name not in config:
            return {
                "success": False,
                "error": f"MCP server '{name}' not found",
                "config": config
            }
        
        del config[name]
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Removed MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

@tool("list_mcp_servers")
def list_mcp_servers() -> Dict[str, Any]:
    """
    List all configured MCP servers.
    
    Returns:
        Dict with success status and list of servers
    """
    try:
        config = load_mcp_config()
        return {
            "success": True,
            "servers": list(config.keys()),
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

In [34]:
from langchain_core.messages import HumanMessage
from experiments.utils.agent_factory import create_agent_with_valid_tools

tools = [add_github_repository_as_mcp_tool, add_mcp_server_tool, remove_mcp_server_tool, list_mcp_servers, git_clone_tool]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools,
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)

"""
stream = agent.astream({"messages": [HumanMessage("Can you clone the github repository: https://github.com/simonskodt/arch-reconstruct-ai")]})
async for chunk in stream:
    print(chunk)


result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)

result = await agent.ainvoke({"messages": [HumanMessage("Can you take the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, and make it into a MCP server")]})
print(result)


client = create_mcp_client_from_config()
mcp_tools = await client.get_tools() 
tools += mcp_tools

[print(tool.name) for tool in tools]
"""

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools, # Tools cannot be dynamically  or hot reloaded?, agent has to be recreated  
    prompt="""#Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)
# result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
# print(result)



### Extract repository details gitingest

In [ ]:
import json
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil

from gitingest import ingest, ingest_async

@tool("extract_repository_details")
async def extract_repository_details(
    local_repository_path: Optional[str],
    github_url: Optional[str] = None,
    output_path: Optional[str] = "extract_repository_details.json",
) -> Dict[str, Any]:
    """
    Extract and ingest a Git repository (local or remote) into a readable LLM format.

    Args:
        local_repository_path: Path to a local repository directory.
        github_url: HTTPS URL of a remote GitHub repository.
        output_path: Output path for the extraction (default: "extract_repository_details.json", use "-" or "stdout" for stdout).
    Returns:
        path to a JSON dict with summary (str), tree (str), and content (str) of the repository.
    """

    try:        
        exclude_patterns = {
            "*.pyc",
            "__pycache__",
            ".git",
            ".venv",
            "venv",
            "env",
            "node_modules",
            ".DS_Store",
            "*.log",
            ".pytest_cache",
            "*.egg-info",
            "dist",
            "build",
            "*.lock",
            ".pylintrc"
        }
        if local_repository_path:
            summary, tree, content = await ingest_async(local_repository_path, exclude_patterns=exclude_patterns, output=output_path)
        elif github_url:
            summary, tree, content = await ingest_async(github_url, exclude_patterns=exclude_patterns, output=output_path)
        else:
            return {"success": False, "error": "Either local_repository_path or github_url must be provided"}

        extraction = {"summary": summary, "tree": tree, "content": content} 

        if output_path is not None:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(extraction, f, ensure_ascii=False, indent=2)
            return {"success": True, "path": output_path}
        return {"success": True, "data": extraction}
    except Exception as e:
        return {"success": False, "error": str(e)}
    
@tool("load_extracted_repository")
def load_extracted_repository(path: str) -> Dict[str, Any]:
    """
    Load the extracted repository details from a file.
    This file is expected to be generated by the `extract_repository_details` tool.
    
    Args:
        path: Path to the file containing the extracted repository details.

    Returns:
        A dict with the loaded repository details.
          -with summary (str), tree (str), and content (str) of the repository.
    """
    if not os.path.exists(path):
        return {"success": False, "error": "File not found"}

    with open(path, "r") as f:
        data = f.read()

    return {"success": True, "data": data}

In [ ]:
tool_input = {
    "local_repository_path": ".",
    "output_path": "details.json",
    "github_url": None,
}

print(extract_repository_details.name)

output = await extract_repository_details.ainvoke(tool_input)

for key in output:
    if key != "content":  
        print(f"{key}:  {output[key]}")
os.getcwd()


tree = output.get("data", {}).get("tree", "")

extract_repository_details
success:  True
path:  details.json
success:  True
path:  details.json


'C:\\Users\\thoma\\Desktop\\arch-reconstruct-ai\\experiments\\tool_calling\\repositories\\zeeguu_api'

## ArchLens

In [38]:
from typing import Optional, Dict, Any, List
from logging import Handler
from pydantic import BaseModel


class ArchLensConfig(BaseModel):
  name: str
  rootFolder: str
  views: Dict[str, Dict[str, List[Dict[str, Any]]]]
  saveLocation:str  = "./diagrams/"

#create Archlensobject example
viewsJson = {"top-level-view-depth-1": {
      "packages": [
        {
          "path": "*",
          "depth": 1
        }
      ]
    },
    "top-level-view-depth-2": {
      "packages": [
        {
          "path": "*",
          "depth": 2
        }
      ]
    }}

archlensObject = ArchLensConfig(name="tets", rootFolder='zeeguu', views=viewsJson)

print(archlensObject)



name='tets' rootFolder='zeeguu' views={'top-level-view-depth-1': {'packages': [{'path': '*', 'depth': 1}]}, 'top-level-view-depth-2': {'packages': [{'path': '*', 'depth': 2}]}} saveLocation='./diagrams/'


In [ ]:
import json
from pathlib import Path

@tool('init_archLens')
def init_archLens(repo_url: str):
    """"If you have cloned the arch-reconstruct-ai repository, initialize archLens."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    if os.path.exists("archlens.json"):
        print("archlens.json already exists, skipping initialization.")
    else:
        os.system("archlens init")

    return "Initialized archLens"

@tool('run_archLens')
def run_archLens(repo_url: str):
    """"Run archLens on the cloned repository."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")
    
    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)
      
    if not os.path.exists("archlens.json"):
        return "archlens.json does not exist. Please run init_archLens first."

    os.system(f"archlens render")

    return "Ran archLens"


@tool('read_archLens_config_file')
def read_archLens_config_file(repo_url: str):
    """"Reads the content of the archlens.json file."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    config_path = "archlens.json"
    if not os.path.exists(config_path):
        return "archlens.json does not exist. Please run init_archLens first."
    
    with open('archlens.json', 'r') as file:
        data = json.load(file)

    print(json.dumps(data, indent=4))
    return "Read config file"


@tool('write_archLens_config_file')
def write_archLens_config_file(repo_url: str, arch: ArchLensConfig):
    """"Writes content to the archlens.json file.
        args: 
    """
    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    config_path = "archlens.json"
    with open(config_path, 'w') as file:
        json.dump(arch.__dict__, file)
    
    return "Wrote to config file"

@tool('create_ArchLensConfig_Object')
def create_ArchLensConfig_Object(packageName:str, path: str, depth: int) -> ArchLensConfig:
    """"Creates an ArchLensConfig object, which is used when writing to the archlens.json file. This is the structure of the ArchLensConfig object:
    viewsJson = {"top-level-view-depth-1": {
      "packages": [
        {
          "path": "*",
          "depth": 1
        }
      ]
    },
    "top-level-view-depth-2": {
      "packages": [
        {
          "path": "*",
          "depth": 2
        }
      ]
    }}
    """

    viewsJson = {packageName: {
      "packages": [
        {
          "path": path,
          "depth": depth
        }
      ]
    }
    }

    archlensObject = ArchLensConfig(name='testing' ,rootFolder='zeeguu', views=viewsJson)
    return archlensObject

@tool('add_view_to_ArchLensConfig_Object')
def add_view_to_ArchLensConfig_Object(archlensObject: ArchLensConfig, packageName:str, path:str, depth: int) -> ArchLensConfig:

  """"Adds a view to an existing ArchLensConfig object. 
          args: 
              archlensObject: The existing ArchLensConfig object.
              name: The name of the view to add.
  """

  archlensObject.views[packageName] = {
      "packages": [
        {
          "path": path,
          "depth": depth
        }
      ]
    }
  return archlensObject


# archConfig = create_ArchLensConfig_Object({"packageName": "top-level-view-depth-1", "path": "*", "depth": 1})
# ar = add_view_to_ArchLensConfig_Object({"archlensObject": archConfig, "packageName": "top-level-view-depth-2", "path": "*", "depth": 2})
# write_archLens_config_file({"repo_url": "https://github.com/zeeguu/api.git", "arch": ar})

# run_archLens('https://github.com/zeeguu/api.git')


In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    "openai:gpt-5-nano",
    tools=[init_archLens, run_archLens, read_archLens_config_file, write_archLens_config_file, create_ArchLensConfig_Object],
    prompt=f"Act as an assistant, that does architectural reconstruction of software repositories. Archlens is a tool which can be used for python repositories",
)

#result = agent.invoke({"messages": [HumanMessage("Can you clone the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, feel free to overwrite if a clone already exists, ")]})
result = agent.invoke({"messages": [HumanMessage("I want you to create an ArchLensConfig object with the name agent, depth 3 and path *. Afterwards write the generated ArchlensConfig object to the archlens.json file and run archlens render. The repository url is https://github.com/zeeguu/api.git")]})
print(result)


## Navigation tools

### List current directory

In [55]:
import os

@tool("List_files__directory")
def list_files_directory() -> list[str]:
    """List the files of the current directory."""
    return os.listdir(".")

### Current working directory

In [41]:
@tool("current_working_directory")
def current_working_directory() -> str:
    """Return the current working directory."""
    return os.getcwd()

### Get parent and child directory

In [42]:
from typing import Any 
@tool("get_child_directory")
def get_child_directory(ls: bool = True) -> dict[str, Any]:
    """Return the child directory of the current working directory."""
    current_dir = os.getcwd()
    child_dir = os.path.join(current_dir, "child")
    dict_result: dict[str, Any] = {"child": child_dir}
    if ls:
        dict_result["child_contents"] = os.listdir(child_dir) if os.path.exists(child_dir) else []

    return dict_result

### Change directory

In [107]:
import os
from pathlib import Path
def get_root_agent_directory() -> Path:
    """Get root directory from state, with fallback to environment variable."""
    root_dir = os.getenv("REPO_BASE_PATH")
    if root_dir:
        return normalize_path(root_dir)
    raise ValueError("REPO_BASE_PATH is not set in the .env file.")


def normalize_path(path: str) -> Path:
    """Normalize a path across OSes and WSL/Windows interop."""
    path = str(path).strip()

    # Convert WSL paths (/mnt/c/...) -> Windows (C:\...)
    if path.startswith("/mnt/") and len(path) > 6 and path[5].isalpha() and path[6] == "/":
        drive = path[5].upper()
        rest = path[7:]
        path = f"{drive}:/{rest}"

    # Convert Git Bash /c/... -> Windows C:/...
    elif path.startswith("/") and len(path) > 2 and path[1].isalpha() and path[2] == "/":
        drive = path[1].upper()
        rest = path[3:]
        path = f"{drive}:/{rest}"

    return Path(path).resolve()


def is_within_repository_directory(path: str) -> bool:
    """Check if path is within the repository directory (including the root itself)."""
    repository_directory = normalize_path(str(get_root_agent_directory())).resolve()
    abs_path = normalize_path(path).resolve()

    # Direct equality (root itself)
    if abs_path == repository_directory:
        return True
    
    # Check if abs_path is a subpath of repository_directory
    try:
        abs_path.relative_to(repository_directory)
        return True
    except ValueError:
        return False



def get_normalized_repository_path(repo_name: str) -> Path:
    """Get the full path to a repository by name."""    
    repository_directory = get_root_agent_directory()
    base_path = Path(repository_directory)
    
    # Include repo_name in path if it's a valid, non-empty name
    if repo_name and repo_name.strip() not in (""):
        return normalize_path(str( (base_path / repo_name).resolve() ))
    
    return normalize_path(str(base_path.resolve()))

@tool("Change_directory")
def change_directory(path: str) -> str:
    """Change the current working directory with improved path handling."""
    try:
        # Get current working directory as base
        current_dir = os.getcwd()
        
        # Handle different path formats
        if os.path.isabs(path):
            target_path = path
        elif path.startswith('./') or path.startswith('../'):
            target_path = os.path.normpath(os.path.join(current_dir, path))
        else:
            # Try as relative to current directory first
            if os.path.exists(os.path.join(current_dir, path)):
                target_path = os.path.normpath(os.path.join(current_dir, path))
            else:
                # Try in repositories folder
                repos_path = get_normalized_repository_path(path)
                if os.path.exists(repos_path):
                    target_path = os.path.normpath(repos_path)
                else:
                    target_path = os.path.normpath(os.path.join(current_dir, path))
        
        # Check if directory exists
        if not is_within_repository_directory(target_path):
            return f"Error: Access to '{target_path}' is outside the allowed repository directory."
        
        if not os.path.exists(target_path):
            return f"Error: Directory '{target_path}' does not exist."
        
        if not os.path.isdir(target_path):
            return f"Error: '{target_path}' is not a directory."
        
        # Change directory
        os.chdir(target_path)
        new_cwd = os.getcwd()
        return f"Successfully changed to: {new_cwd}"
        
    except Exception as e:
        return f"Error changing directory: {str(e)}"

@tool("navigate_to_repository")
def navigate_to_repository(repo_name: str) -> str:
    """Navigate directly to a specific repository in the repositories folder."""
    try:
        repository_path = get_normalized_repository_path(repo_name)  

        if not os.path.exists(repository_path):
            return f"Error: Repository '{repo_name}' not found at {repository_path}"
        
        if not os.path.isdir(repository_path):
            return f"Error: '{repository_path}' is not a directory."
        
        os.chdir(repository_path)
        return f"Successfully navigated to repository: {os.getcwd()}"
        
    except Exception as e:
        return f"Error navigating to repository: {str(e)}"

@tool("list_repositories")
def list_repositories() -> str:
    """List all available repositories in the repositories folder."""
    try:
        repositories_path = get_normalized_repository_path("")  # Get path to repositories folder
        
        if not os.path.exists(repositories_path.resolve()):
            return f"Error: Repositories folder not found at {repositories_path}"

        repos = [repo_dir for repo_dir in os.listdir(repositories_path)
                if os.path.isdir(os.path.join(repositories_path, repo_dir))]
        if repos:
            return f"Available repositories: {', '.join(repos)}"
        else:
            return "No repositories found in the repositories folder."
            
    except Exception as e:
        return f"Error listing repositories: {str(e)}"

### Adjust archLens tools to run in local path

In [90]:
@tool('run_archLens')
def run_archLens() -> str:
    """"Run archLens on the current directory (should be in a repository)."""
    current_dir = os.getcwd()
    
    if not os.path.exists("archlens.json"):
        return f"archlens.json does not exist in {current_dir}. Please make sure you're in a repository directory and run init_archLens first."

    try:
        exit_code = os.system("archlens render")
        if exit_code == 0:
            return f"Successfully ran archLens in {current_dir}"
        else:
            return f"archLens render failed with exit code {exit_code}"
    except Exception as e:
        return f"Error running archLens: {str(e)}"

@tool('init_archLens')
def init_archLens() -> str:
    """"Initialize archLens in the current directory (should be in a repository)."""
    current_dir = os.getcwd()
    
    # Check if we're in what looks like a repository directory
    if not (REPOSITORY_FOLDER in current_dir or os.path.exists(".git")):
        return f"Current directory ({current_dir}) doesn't appear to be a repository. Please navigate to a repository first."
    
    if os.path.exists("archlens.json"):
        return f"archlens.json already exists in {current_dir}, skipping initialization."
    else:
        try:
            exit_code = os.system("archlens init")
            if exit_code == 0:
                return f"Successfully initialized archLens in {current_dir}"
            else:
                return f"archLens init failed with exit code {exit_code}"
        except Exception as e:
            return f"Error initializing archLens: {str(e)}"

### Navigation example

In [121]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import asyncio
load_dotenv()


# Test path normalization
root_dir = get_root_agent_directory()
print(f"Root directory: {root_dir}")
root_dir = str(get_normalized_repository_path(""))
print(f"Normalized root dir: {root_dir}")

print(f"Is root dir valid? {is_within_repository_directory(root_dir)}")
print(f"Is subdir valid? {is_within_repository_directory("zeeguu_api")}")
print(f"Is dir valid? {is_within_repository_directory(("/Desktop/arch-reconstruct-ai/experiments/tool_calling  /repositories/zeeguu_api"))}")
print(f"Is dir valid? {is_within_repository_directory(('/etc/passwd'))}")

## clone repository for testing
tool_input = {
    "repo_url": "https://github.com/simonskodt/arch-reconstruct-ai",
    "dest": "arch-reconstruct-ai",
    "overwrite": False
}

if False:  # Enable cloning for testing
    print(f"\nCloning repository...")

    output = git_clone_tool.invoke(tool_input)
    print(f"Clone result: {output}")

# Update tools list to include the new navigation tool
tools = [
    # git_clone_tool, 
    
    list_repositories,
    # navigate_to_repository,
    
    change_directory, 
    # list_files_directory, 
    current_working_directory, 
    # get_child_directory, 

    run_archLens, 
    init_archLens, 

    extract_repository_details, 
    load_extracted_repository
]

tool_descriptions = [tool.description for tool in tools]

agent = create_agent(
    "openai:gpt-5-nano",
    tools=tools,
    prompt=f"Act as an assistant that can navigate the file system using the tools provided.\
      The available tools are: {tool_descriptions}",
)

print(f"\nTesting agent navigation...")

# Test navigation commands
navigation_tests = [
    # "Can you list the current directory?",
    # "Can you list all available repositories?", 
    # "Can you run archLens in root of zeeguu_api repository?",

    "Can you navigate to the zeeguu_api diagrams directory?",
    # "Can you run archLens in root of arch-reconstruct-ai repository?",
    # "Can you list the current directory after navigation?",
    # "Can you show me the current working directory path?",
    # "Can you initialize archLens in arch-reconstruct-ai repository?",
    # "Can you list all available repositories?",     
    # "Can you extract the repository details of arch-reconstruct-ai repository?",
    # "Can you show me the summary and tree of the extracted repository details?"
]

print(os.getcwd())

for i, test_query in enumerate(navigation_tests):
    print(f"\n--- Test {i+1}: {test_query} ---")
    try:
        result = await agent.ainvoke({"messages": [HumanMessage(test_query)]})
        
        # Print the last message (agent's response)
        if result.get("messages"):
            last_message = result["messages"][-1]
            print(f"Response: {last_message.content}")
        else:
            print("No response received")
    except Exception as e:
        print(f"Error: {e}")
    
    # Small delay between tests
    await asyncio.sleep(5)

Root directory: C:\Users\thoma\Desktop\arch-reconstruct-ai\experiments\tool_calling\repositories
Normalized root dir: C:\Users\thoma\Desktop\arch-reconstruct-ai\experiments\tool_calling\repositories
Is root dir valid? True
Is subdir valid? True
Is dir valid? False
Is dir valid? False

Testing agent navigation...
C:\Users\thoma\Desktop\arch-reconstruct-ai\experiments\tool_calling\repositories\arch-reconstruct-ai

--- Test 1: Can you navigate to the root of zeeguu_api diagrams directory? ---
Response: I’ve navigated to the root of the zeeguu_api diagrams directory:

Current path:
C:\Users\thoma\Desktop\arch-reconstruct-ai\experiments\tool_calling\repositories\zeeguu_api\diagrams

Would you like me to open a specific file, list contents, or do something else in this directory?
